## Chi-square

Compute chi-squared stats between each non-negative feature and class. 

- This score should be used to evaluate categorical variables in a classification task.

In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split

# to determine the chi2 value
from sklearn.feature_selection import chi2

# to select the features
from sklearn.feature_selection import SelectKBest

In [2]:
# load dataset

data = pd.read_csv('../titanic.csv')
data.shape

(1306, 9)

In [3]:
data.head()

,pclass,survived,sex,age,sibsp,parch,fare,cabin,embarked
0,1,1,female,29.0000,0,0,211.3375,B5,S
1,1,1,male,0.9167,1,2,151.5500,C22,S
2,1,0,female,2.0000,1,2,151.5500,C22,S
3,1,0,male,30.0000,1,2,151.5500,C22,S
4,1,0,female,25.0000,1,2,151.5500,C22,S


In [4]:
# the categorical variables in the titanic are pclass, sex and embarked

# first I will encode the labels of the categories into numbers

# for Sex / Gender
data['sex'] = np.where(data['sex'] == 'male', 1, 0)

# for Embarked
ordinal_label = {k: i for i, k in enumerate(data['embarked'].unique(), 0)}
data['embarked'] = data['embarked'].map(ordinal_label)

# pclass is already ordinal

**Important**

In all feature selection procedures, it is good practice to select the features by examining only the training set. And this is to avoid overfit.

In [5]:
# separate train and test sets
X_train, X_test, y_train, y_test = train_test_split(
    data[['pclass', 'sex', 'embarked']],
    data['survived'],
    test_size=0.3,
    random_state=0)

X_train.shape, X_test.shape

((914, 3), (392, 3))

In [6]:
# calculate the chi2 p_value between each of the variables
# and the target

# chi2 returns 2 arrays, one contains the F-Scores which are then
# evaluated against the chi2 distribution to obtain the pvalue.
# The pvalues are in the second array

f_score = chi2(X_train.fillna(0), y_train)

# the 2 arrays of values
f_score

(array([32.49351818, 81.97868872,  5.66498574]),
 array([1.19590735e-08, 1.37561991e-19, 1.73068577e-02]))

In [7]:
# 1) let's capture the p_values (in the second array, remember python indexes at 0) in a pandas Series
# 2) add the variable names in the index
# 3) order the variables based on their fscore

pvalues = pd.Series(f_score[1])
pvalues.index = X_train.columns
pvalues.sort_values(ascending=True)

sex         1.375620e-19
pclass      1.195907e-08
embarked    1.730686e-02
dtype: float64

Contrarily to MI, where we were interested in the higher MI values, for the chi2, the smaller the p_value the more significant the feature is to predict the target.

Thus, from the result above, Sex is the most important feature, as it has the smallest p-value.

In this demo, we used chi2 to determine the predictive value of 3 categorical variables only. If the dataset contained several categorical variables, we could then combine this procedure with SelectKBest or SelectPercentile, as we did in the previous notebook, to select the top k features, or the features in the top n percentile, based on the chi2 p-values.

Let's select the top 1 feature for the demo:

In [8]:
sel_ = SelectKBest(chi2, k=1).fit(X_train, y_train)

# display features
X_train.columns[sel_.get_support()]

Index(['sex'], dtype='object')

In [9]:
# to remove the rest of the features:

X_train = sel_.transform(X_train)
X_test = sel_.transform(X_test)

That is all for this lecture, I hope you enjoyed it and see you in the next one!